![](https://storage.googleapis.com/kaggle-datasets-images/312121/636393/a5097396fc07cf882d3e0d631b100a36/dataset-cover.jpg?t=2019-08-23-15-00-53)

***

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
The Dataset is fully dedicated for the developers who want to train the model on Weather Forecasting for Indian climate.<br><br> This dataset provides data from 1st January 2013 to 24th April 2017 in the city of Delhi, India. <br><br>The 4 parameters here are
meantemp, humidity, wind_speed, meanpressure.
    
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 
    
5 columns in the dataset, below is <u>description of features</u>:

*  (1) <b>meantemp</b>: Mean temperature averaged out from multiple 3 hour intervals in a day. 
    
*  (2) <b>humidity</b>: Humidity value for the day (units are grams of water vapor per cubic meter volume of air).
    
    
*  (3) <b>wind_speed</b>: Wind speed measured in kmph.
    
    
*  (4) <b>meanpressure</b>: Pressure reading of weather (measure in atm)
    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using LSTM, Pytorch. <br><br>
    The Target Column used here is <b>meantemp</b>
</h4>

 

https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [ ]:
import os
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [ ]:
import pandas  as pd
train_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTrain.csv')
test_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTest.csv')
train_df.head()

<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [ ]:
print('Train set \n\n')
train_df.head()

In [ ]:
from datetime import datetime

def get_day_of_year(date):
    date_obj = datetime.strptime(date, '%Y-%m-%d')  # Ensure your date format matches '%Y-%m-%d'
    return date_obj.timetuple().tm_yday

# Apply function to get the day of the year
train_df['day_of_year'] = train_df['date'].apply(get_day_of_year)
test_df['day_of_year'] = test_df['date'].apply(get_day_of_year)

# # Drop the original date column
train_df = train_df.drop('date', axis=1)
test_df = test_df.drop('date', axis=1)

# Display the first few rows to verify
print(train_df.head())
print(test_df.head())


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Assuming train_df and test_df are pandas DataFrames

# Initialize scalers
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

# Apply StandardScaler to all but the last column
train_df.iloc[:, :-1] = standard_scaler.fit_transform(train_df.iloc[:, :-1])
test_df.iloc[:, :-1] = standard_scaler.transform(test_df.iloc[:, :-1])

# Apply MinMaxScaler to the last column
# Note: .values.reshape(-1, 1) ensures the data is in the right shape for fitting
train_df.iloc[:, -1] = min_max_scaler.fit_transform(train_df.iloc[:, -1].values.reshape(-1, 1))
test_df.iloc[:, -1] = min_max_scaler.transform(test_df.iloc[:, -1].values.reshape(-1, 1))

# Verification of the number of features
num_features = train_df.shape[1]

In [ ]:
import torch
from torch.utils.data import Dataset
import random

class SequentialDataset(Dataset):
    def __init__(self, df, indices, max_window_size, num_classes):
        self.df = df
        self.indices = indices
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2
        self.num_classes = num_classes

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        start_idx = self.indices[idx]
        window_size = random.randint(self.min_window_size, self.max_window_size)
        end_idx = min(start_idx + window_size, len(self.df))

        seq = self.df.iloc[start_idx:end_idx]
        X, y = seq.values[:-1, :], seq.values[1:, :]

        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

In [ ]:
from random import shuffle

# Assume 'df' is your DataFrame and 'event' is the column containing labels

def generate_indices(df, max_window_size):
    indices = []
    max_index = len(df) - max_window_size + 1  # Calculate the maximum starting index
    
    for i in range(max_index):
        # Check if all labels in the window are the same
        indices.append(i)
    
    return indices

# Assuming 'df' and 'num_classes' are defined
max_window_size = 14
train_indices = generate_indices(train_df, max_window_size)
test_indices = generate_indices(test_df, max_window_size)

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

trainset = SequentialDataset(df=train_df, indices=train_indices, max_window_size=max_window_size, num_classes=num_features)
testset = SequentialDataset(df=test_df, indices=test_indices, max_window_size=max_window_size, num_classes=num_features)

<a id="4"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>4.</b> Modeling</b></h1>

<b> First, define Modeling Class 'LSTMModel'</b>

In [ ]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

data_config = DataConfig(dataset_name = 'daily-delhi-climate', task_type='regression', obs_shape=[num_features], label_size=num_features)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'gpt', encoder_model = 'none')

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False) 

In [ ]:
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

# Standardizing the features (excluding the class column)
sc = StandardScaler()
df.iloc[:, :-1] = sc.fit_transform(df.iloc[:, :-1])

# Total elements including features and class
n_elements = df.shape[1]
print(f"Number of features and classes (total columns): {n_elements}")

Number of features and classes (total columns): 31


In [ ]:
trainer_hub.train(trainset, testset)